In [16]:
import random

In [24]:
def logging_time(original_fn):
    import time
    from functools import wraps

    @wraps(original_fn)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = original_fn(*args, **kwargs)

        end_time = time.time()
        print("WorkingTime[{}]: {} sec".format(original_fn.__name__, end_time - start_time))
        return result
    return wrapper

## Environment

In [17]:
class GridWorld():
    def __init__(self): # 초기 위치 선정
        self.x=0
        self.y=0
    
    def step(self, a): # 액션에 따른 상태 변이, 보상 정의, 종결 여부 판단
        # 4가지 action - 상하좌우
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1 # 보상은 항상 -1
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_right(self): # 이동
        self.y += 1  
        if self.y > 3:
            self.y = 3
      
    def move_left(self): # 이동
        self.y -= 1
        if self.y < 0:
            self.y = 0
      
    def move_up(self): # 이동
        self.x -= 1
        if self.x < 0:
            self.x = 0
  
    def move_down(self): # 이동
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self): # 에피소드가 끝났는지 판단
        if self.x == 3 and self.y == 3:
            return True
        else :
            return False

    def get_state(self): # 현재 위치 반환
        return (self.x, self.y)
      
    def reset(self): # 초기화
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## Agent

In [18]:
class Agent():
    def __init__(self):
        pass

    def select_action(self): # 랜덤하게 액션을 선택하는 정책
        coin = random.random()

        # 확률 분포의 형태
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

## MC 실행

In [25]:
@logging_time
def main():
    env = GridWorld()
    agent = Agent()
    data = [[0] * 4 for _ in range(4)] # 4x4 크기의 0으로 채워진 2차원 리스트 생성
    gamma = 1.0
    alpha = 0.0001

    for _ in range(50000) : # 총 50000번의 에피소드 진행
        done = False
        history = []
        while not done:
            action = agent.select_action() # 정책에 따라 액션 선택
            (x,y), reward, done = env.step(action) # 액션에 따른 상태 변이
            history.append((x,y,reward)) # 에피소드의 상태, 보상 정보 저장
        env.reset() # 에피소드가 끝나면 환경 초기화

        # 매 에피소드가 끝나고 바로 해당 데이터를 이용해 테이블을 업데이트
        cum_reward = 0
        for transition in history[::-1]:
            # 방문했던 상태들을 뒤에서부터 보며 차례차례 리턴을 계산
            x, y, reward = transition
            data[x][y] = data[x][y] + alpha * (cum_reward - data[x][y])
            cum_reward = reward + gamma * cum_reward
    
    return data

In [26]:
MC_result = main()
# 학습이 끝나고 난 후 데이터를 출력해보기 위한 코드
for row in MC_result:
    print(row)

WorkingTime[main]: 1.5207290649414062 sec
[-58.07162999147687, -55.96706734885653, -52.63755035941325, -50.58554798280284]
[-56.5626315066358, -53.2992747179113, -48.39964468256098, -43.50603921126412]
[-53.72415077745313, -48.5189293936831, -39.57823200618536, -28.741587721729935]
[-51.93925750969205, -44.38532270969551, -29.51678797451844, 0.0]
